In [ ]:
pip install openai


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')

In [ ]:
MODEL = "gpt-4o-2024-08-06"

In [ ]:
import openai

# optional; defaults to `os.environ['OPENAI_API_KEY']`
openai.api_key = os.environ["OPENAI_API_KEY"]

# all client options can be configured just like the `OpenAI` instantiation counterpart
openai.base_url = "https://api.openai-hk.com/v1/"
openai.default_headers = {"x-foo": "true"}

completion = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "How do I output all files in a directory using Python?",
        },
    ],
)
print(completion.choices[0].message.content)

To output (or list) all files in a directory using Python, you can use the `os` module or the `pathlib` module. Here are examples of both methods:

### Using `os` module

```python
import os

# Specify the directory you want to list files from
directory = '/path/to/directory'

# List all files in the specified directory
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)  # Get the full path
    if os.path.isfile(file_path):  # Check if it's a file
        print(filename)
```

### Using `pathlib` module (Python 3.4 and above)

```python
from pathlib import Path

# Specify the directory
directory = Path('/path/to/directory')

# List all files in the directory
for file in directory.iterdir():
    if file.is_file():  # Check if it's a file
        print(file.name)
```

### Notes:
- Replace `'/path/to/directory'` with the path to the directory you want to inspect.
- Both methods will only list the files directly in the specified directory and will not r

In [ ]:
def get_math_solution(question):
  response = openai.chat.completions.create(
  model="gpt-4o-2024-08-06",
  messages=[
      {
          "role": "system",
          "content": math_tutor_prompt
      },
      {
          "role": "user",
          "content": question
      }
  ],
  response_format={
      "type": "json_schema",
      "json_schema": {
          "name": "math_reasoning",
          "schema": {
              "type": "object",
              "properties": {
                  "steps": {
                      "type": "array",
                      "items": {
                          "type": "object",
                          "properties": {
                              "explanation": {"type": "string"},
                              "output": {"type": "string"}
                          },
                          "required": ["explanation", "output"],
                          "additionalProperties": False
                      }
                  },
                  "final_answer": {"type": "string"}
              },
              "required": ["steps", "final_answer"],
              "additionalProperties": False
          },
          "strict": True
      }
  }
  )

  return response.choices[0].message

In [ ]:
# Testing with an example question
math_tutor_prompt = '''
    You are a helpful math tutor. You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
'''
question = "how can I solve 8x + 7 = -23"

result = get_math_solution(question)

print(result.content)

BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'response_format.json_schema'. (request id: 20240807110554530054185fBUYkNgZ) (request id: 20240807110554453586051H1hdc4tq)", 'type': 'invalid_request_error', 'param': 'response_format.json_schema', 'code': 'missing_required_parameter'}}

In [ ]:
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

def get_math_solution(question: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": math_tutor_prompt},
            {"role": "user", "content": question},
        ],
        response_format=MathReasoning,
    )

    return completion.choices[0].message

In [ ]:
result = get_math_solution(question).parsed
print(result.steps)
print("Final answer:")
print(result.final_answer)

APIConnectionError: Connection error.